# Fibonacci Retracement Strategy

### Load the data

In [10]:
import pandas as pd
import pandas_ta as ta
#df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")
df = pd.read_csv("test_EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, length=12)
df['EMA'] = ta.ema(df.close, length=150)
df.tail()

df=df[0:2000]

### Trend detection

In [11]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

Fibonacci Signal

In [12]:
def generate_signal(df,l,backcandles, gap, zone_threshold, price_diff_threshold):
    max_price = df.high[l-backcandles:l-gap].max()
    min_price = df.low[l-backcandles:l-gap].min()
    index_max = df.high[l-backcandles:l-gap].idxmax()
    index_min = df.low[l-backcandles:l-gap].idxmin()
    price_diff = max_price - min_price

    if (df.EMASignal[l] == 2 
        and (index_min < index_max) 
        and price_diff>price_diff_threshold): 
        l1 = max_price - 0.62 * price_diff # position entry 0.62
        l2 = max_price - 0.78 * price_diff # SL 0.78
        l3 = max_price - 0. * price_diff # TP
        if abs(df.close[l]-l1) < zone_threshold and df.high[l-gap:l].min()>l1:
            return (2, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)
        
    elif (df.EMASignal[l] == 1 
          and (index_min > index_max) 
          and price_diff>price_diff_threshold): 
        l1 = min_price + 0.62 * price_diff # position entry 0.62
        l2 = min_price + 0.78 * price_diff # SL 0.78
        l3 = min_price + 0. * price_diff # TP
        if abs(df.close[l]-l1) < zone_threshold and df.low[l-gap:l].max()<l1:
            return (1, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)
    
    else:
        return (0,0,0,0,0)


gap_candles = 5
backcandles = 40
signal = [0 for i in range(len(df))]
TP = [0 for i in range(len(df))]
SL = [0 for i in range(len(df))]
MinSwing = [0 for i in range(len(df))]
MaxSwing = [0 for i in range(len(df))]

for row in range(backcandles, len(df)):
    gen_sig = generate_signal(df, row, backcandles=backcandles, gap=gap_candles, zone_threshold=0.001, price_diff_threshold=0.01)
    signal[row] = gen_sig[0]
    SL[row] = gen_sig[1]
    TP[row] = gen_sig[2]
    MinSwing[row] = gen_sig[3]
    MaxSwing[row] = gen_sig[4]
    
df['signal'] = signal
df['SL'] = SL
df['TP'] = TP
df['MinSwing'] = MinSwing
df['MaxSwing'] = MaxSwing


In [13]:
df[df.signal !=0][:10]
#df.head(50)

,Gmt time,open,high,low,close,volume,RSI,EMA,EMASignal,signal,SL,TP,MinSwing,MaxSwing
149,13.05.2003 02:00:00.000,1.14817,1.15183,1.14747,1.15063,2.787880e+07,39.526187,1.140474,2,2,1.147603,1.16229,113,132
150,13.05.2003 03:00:00.000,1.15045,1.15128,1.14931,1.15034,2.537580e+07,38.863748,1.140605,2,2,1.147603,1.16229,113,132
151,13.05.2003 04:00:00.000,1.15043,1.15090,1.14898,1.14964,2.864920e+07,37.221120,1.140724,2,2,1.147603,1.16229,113,132
275,20.05.2003 08:00:00.000,1.16389,1.16490,1.16103,1.16156,2.718980e+07,37.949316,1.153291,2,2,1.157228,1.17353,237,252
276,20.05.2003 09:00:00.000,1.16169,1.16328,1.16062,1.16062,2.705410e+07,35.494704,1.153388,2,2,1.157228,1.17353,237,252
659,11.06.2003 09:00:00.000,1.17032,1.17245,1.16985,1.17216,2.637310e+07,64.461611,1.172434,1,1,1.173549,1.16571,652,621
813,19.06.2003 19:00:00.000,1.17268,1.17289,1.17057,1.17122,1.048652e+08,59.890142,1.174544,1,1,1.175342,1.15826,797,774
814,19.06.2003 20:00:00.000,1.17092,1.17226,1.17092,1.17173,6.485020e+07,60.958667,1.174507,1,1,1.175342,1.15826,797,774
876,24.06.2003 10:00:00.000,1.15572,1.15801,1.15524,1.15673,2.730180e+07,57.551262,1.165611,1,1,1.159005,1.15069,857,838
877,24.06.2003 11:00:00.000,1.15695,1.15760,1.15597,1.15647,2.778290e+07,55.942973,1.165489,1,1,1.159005,1.15069,857,838


In [14]:
import numpy as np
def pointpos(x):
    if x['signal']==1:
        return x['high']+1e-4
    elif x['signal']==2:
        return x['low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [15]:
dfpl = df[750:950]
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800, 
    paper_bgcolor='black',
    plot_bgcolor='black')
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=8, color="MediumPurple"),
                name="Signal")
fig.show()

In [16]:
df = df.rename(columns={"open": "Open", "high":"High", "low":"Low", "close": "Close", "volume":"Volume"})
def SIGNAL():
    return df.signal

In [17]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.99 #1000
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
       
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.SL[-1]
            tp1 = self.data.TP[-1]
            tp2 = tp1-(tp1-self.data.Close[-1])/2
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SL[-1]
            tp1 = self.data.TP[-1]
            tp2 = tp1+(self.data.Close[-1]-tp1)/2
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=100, margin=1/100, commission=0.0000)
stat = bt.run()
stat

/tmp/ipykernel_64859/4137819960.py:27: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2999.0
Duration                               2999.0
Exposure Time [%]                   10.366667
Equity Final [$]                    95.039384
Equity Peak [$]                    424.644625
Return [%]                          -4.960616
Buy & Hold Return [%]                4.451271
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -89.327174
Avg. Drawdown [%]                   -31.99919
Max. Drawdown Duration                 2098.0
Avg. Drawdown Duration             401.833333
# Trades                                 37.0
Win Rate [%]                        27.027027
Best Trade [%]                       1.081201
Worst Trade [%]                     -0.365798
Avg. Trade [%]                    

In [18]:
bt.plot()

GridPlot(id='p1638', ...)